# Łączenie modeli i metody zespołowe w uczeniu maszynowym

## Wprowadzenie

Metody zespołowe (ang. "ensembling") to technika łączenia wielu modeli uczenia maszynowego w celu uzyskania lepszych wyników niż pojedynczy model. Idea opiera się na zasadzie "mądrość tłumu" - zespół przeciętnych modeli może przewyższyć jeden doskonały model.

### Dlaczego ensembling ma szanse działać?

- Redukcja wariancji - uśrednianie predykcji zmniejsza ryzyko przeuczenia
-  Redukcja bias - różne modele mogą kompensować swoje słabości
-  Lepsza generalizacja - zespół jest bardziej stabilny na nowych danych
-  Odporność - mniejsza wrażliwość na szum i wartości odstające

### Główne typy modeli zespołowych
1. Bagging (Bootstrap Aggregating)

- Idea: Trenuj wiele modeli na różnych podzbiorach danych (z powtórzeniami)
- Agregacja: Głosowanie (klasyfikacja) lub uśrednianie (regresja)
- Przykłady: Random Forest, Bagging Classifier
- Zalety: Redukuje wariancję, działa dobrze z niestabilnymi modelami
- Kiedy używać: Gdy model ma tendencję do przeuczenia

2. Boosting

- Idea: Trenuj modele sekwencyjnie, każdy kolejny koncentruje się na błędach poprzedniego
- Agregacja: Ważona suma predykcji
- Przykłady: AdaBoost, Gradient Boosting, XGBoost, LightGBM, CatBoost
- Zalety: Redukuje bias i wariancję, często najlepsze wyniki
- Kiedy używać: Gdy potrzebujesz najwyższej dokładności

3. Stacking

- Idea: Trenuj meta-model, który uczy się kombinować predykcje bazowych modeli
- Agregacja: Uczony meta-model
- Zalety: Może łączyć różnorodne modele
- Kiedy używać: Gdy masz czas na eksperymentowanie i chcesz wycisnąć ostatnie procenty wydajności

4. Voting/Blending

- Idea: Prosta kombinacja predykcji (hard/soft voting)
- Agregacja: Głosowanie lub uśrednianie
- Zalety: Prosty, szybki, łatwy do zaimplementowania
- Kiedy używać: Jako pierwszy krok w ensemblingu

## Podstawowe zasady efektywnego ensemblingu

- Różnorodność modeli - modele powinny popełniać różne błędy
- Jakość bazowych modeli - modele nie mogą być zbyt słabe
- Balans bias-variance - łącz modele o różnych charakterystykach
- Walidacja krzyżowa - oceniaj zespół na oddzielnych danych

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Metody ensemble
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier
)

import warnings
warnings.filterwarnings('ignore')

# Generowanie danych
print("="*70)
print("ENSEMBLING DLA KLASYFIKACJI")
print("="*70)

np.random.seed(42)
X, y = make_classification(
    n_samples=2000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=3,
    n_clusters_per_class=2,
    weights=[0.3, 0.3, 0.4],
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nRozmiar zbioru treningowego: {X_train.shape}")
print(f"Rozmiar zbioru testowego: {X_test.shape}")
print(f"Liczba klas: {len(np.unique(y))}")
print(f"Rozkład klas w zbiorze treningowym: {np.bincount(y_train)}")

# ============================================================================
# SEKCJA 1: POJEDYNCZE MODELE BAZOWE
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 1: POJEDYNCZE MODELE BAZOWE")
print("="*70)

base_models = {
    'Drzewo decyzyjne': DecisionTreeClassifier(max_depth=10, random_state=42),
    'Regresja logistyczna': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB()
}

base_results = {}

for name, model in base_models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    base_results[name] = {
        'mean': scores.mean(),
        'std': scores.std()
    }
    print(f"\n{name}:")
    print(f"  CV Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")

# ============================================================================
# SEKCJA 2: BAGGING
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 2: BAGGING - REDUKCJA WARIANCJI")
print("="*70)

# Bagging z Decision Tree
bagging_dt = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=10, random_state=42),
    n_estimators=50,
    max_samples=0.8,
    max_features=0.8,
    bootstrap=True,
    random_state=42
)

scores_bagging = cross_val_score(bagging_dt, X_train, y_train, cv=5, scoring='accuracy')
print(f"\nBagging (drzewo decyzyjne, n=50):")
print(f"  CV Accuracy: {scores_bagging.mean():.4f} (+/- {scores_bagging.std():.4f})")
print(f"  Poprawa vs pojedyncze drzewo: {scores_bagging.mean() - base_results['Drzewo decyzyjne']['mean']:.4f}")

# Random Forest (specjalizowana wersja baggingu)
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    max_features='sqrt',
    random_state=42
)

scores_rf = cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
print(f"\nRandom Forest (n=100):")
print(f"  CV Accuracy: {scores_rf.mean():.4f} (+/- {scores_rf.std():.4f})")

# Analiza wpływu liczby estimatorów
print("\nAnaliza wpływu liczby drzew w Random Forest:")
n_estimators_range = [10, 25, 50, 100, 200]
rf_scores_by_n = []

for n in n_estimators_range:
    rf_temp = RandomForestClassifier(
        n_estimators=n,
        max_depth=10,
        random_state=42
    )
    scores = cross_val_score(rf_temp, X_train, y_train, cv=5, scoring='accuracy')
    rf_scores_by_n.append(scores.mean())
    print(f"  n={n:3d}: {scores.mean():.4f}")

plt.figure(figsize=(10, 6))
plt.plot(n_estimators_range, rf_scores_by_n, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Liczba drzew')
plt.ylabel('CV Accuracy')
plt.title('Wpływ liczby drzew na wydajność Random Forest')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================================
# SEKCJA 3: BOOSTING
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 3: BOOSTING - REDUKCJA BIAS I WARIANCJI")
print("="*70)

# AdaBoost
adaboost = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3),
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)

scores_ada = cross_val_score(adaboost, X_train, y_train, cv=5, scoring='accuracy')
print(f"\nAdaBoost (n=100, lr=1.0):")
print(f"  CV Accuracy: {scores_ada.mean():.4f} (+/- {scores_ada.std():.4f})")

# Gradient Boosting
gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

scores_gb = cross_val_score(gb, X_train, y_train, cv=5, scoring='accuracy')
print(f"\nGradient Boosting (n=100, lr=0.1):")
print(f"  CV Accuracy: {scores_gb.mean():.4f} (+/- {scores_gb.std():.4f})")

# Analiza wpływu learning rate
print("\nAnaliza wpływu learning rate w Gradient Boosting:")
learning_rates = [0.01, 0.05, 0.1, 0.3, 0.5, 1.0]
gb_scores_by_lr = []

for lr in learning_rates:
    gb_temp = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=lr,
        max_depth=3,
        random_state=42
    )
    scores = cross_val_score(gb_temp, X_train, y_train, cv=5, scoring='accuracy')
    gb_scores_by_lr.append(scores.mean())
    print(f"  lr={lr:.2f}: {scores.mean():.4f}")

plt.figure(figsize=(10, 6))
plt.semilogx(learning_rates, gb_scores_by_lr, 'go-', linewidth=2, markersize=8)
plt.xlabel('Learning Rate')
plt.ylabel('CV Accuracy')
plt.title('Wpływ learning rate na wydajność Gradient Boosting')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================================
# SEKCJA 4: VOTING
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 4: VOTING - PROSTA AGREGACJA MODELI")
print("="*70)

# Hard Voting - głosowanie większościowe
voting_hard = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42)),
        ('svm', SVC(kernel='rbf', probability=True, random_state=42))
    ],
    voting='hard'
)

scores_voting_hard = cross_val_score(voting_hard, X_train, y_train, cv=5, scoring='accuracy')
print(f"\nHard Voting (RF + GB + SVM):")
print(f"  CV Accuracy: {scores_voting_hard.mean():.4f} (+/- {scores_voting_hard.std():.4f})")

# Soft Voting - uśrednianie prawdopodobieństw
voting_soft = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42)),
        ('svm', SVC(kernel='rbf', probability=True, random_state=42))
    ],
    voting='soft'
)

scores_voting_soft = cross_val_score(voting_soft, X_train, y_train, cv=5, scoring='accuracy')
print(f"\nSoft Voting (RF + GB + SVM):")
print(f"  CV Accuracy: {scores_voting_soft.mean():.4f} (+/- {scores_voting_soft.std():.4f})")

# ============================================================================
# SEKCJA 5: STACKING
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 5: STACKING - META-MODEL")
print("="*70)

# Definicja modeli bazowych
base_learners = [
    ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42)),
    ('svm', SVC(kernel='rbf', probability=True, random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

# Meta-model - Logistic Regression
stacking = StackingClassifier(
    estimators=base_learners,
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5
)

scores_stacking = cross_val_score(stacking, X_train, y_train, cv=5, scoring='accuracy')
print(f"\nStacking (RF + GB + SVM + KNN → RegLog):")
print(f"  CV Accuracy: {scores_stacking.mean():.4f} (+/- {scores_stacking.std():.4f})")

# ============================================================================
# SEKCJA 6: PORÓWNANIE WSZYSTKICH METOD
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 6: PORÓWNANIE WSZYSTKICH METOD")
print("="*70)

# Zbieranie wyników
all_results = {
    'Drzewo decyzyjne': base_results['Drzewo decyzyjne']['mean'],
    'Random Forest': scores_rf.mean(),
    'Bagging': scores_bagging.mean(),
    'AdaBoost': scores_ada.mean(),
    'Gradient Boosting': scores_gb.mean(),
    'Hard Voting': scores_voting_hard.mean(),
    'Soft Voting': scores_voting_soft.mean(),
    'Stacking': scores_stacking.mean()
}

# Sortowanie wyników
sorted_results = dict(sorted(all_results.items(), key=lambda x: x[1], reverse=True))

print("\nRanking metod (od najlepszej):")
for i, (method, score) in enumerate(sorted_results.items(), 1):
    print(f"{i}. {method:20s}: {score:.4f}")

# Wizualizacja porównania
plt.figure(figsize=(12, 6))
methods = list(sorted_results.keys())
scores = list(sorted_results.values())
colors = ['red' if 'Drzewo' in m else 'green' if any(x in m for x in ['Bagging', 'Random', 'AdaBoost', 'Gradient'])
          else 'blue' for m in methods]

plt.barh(methods, scores, color=colors, alpha=0.7)
plt.xlabel('CV Accuracy')
plt.title('Porównanie metod ensemblingu')
plt.xlim([min(scores) - 0.02, max(scores) + 0.02])
for i, (method, score) in enumerate(zip(methods, scores)):
    plt.text(score + 0.002, i, f'{score:.4f}', va='center', fontweight='bold')
plt.tight_layout()
plt.show()

# ============================================================================
# SEKCJA 7: OCENA NAJLEPSZEGO MODELU NA ZBIORZE TESTOWYM
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 7: OCENA NA ZBIORZE TESTOWYM")
print("="*70)

# Wybierz najlepszy model
best_model_name = list(sorted_results.keys())[0]
print(f"\nNajlepszy model: {best_model_name}")

# Trenuj i testuj najlepszy model (użyjmy Stackingu)
stacking.fit(X_train, y_train)
y_pred = stacking.predict(X_test)

print(f"\nWyniki na zbiorze testowym:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"\nRaport klasyfikacji:\n{classification_report(y_test, y_pred)}")

# Macierz pomyłek
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Macierz pomyłek - {best_model_name}')
plt.ylabel('Prawdziwa klasa')
plt.xlabel('Przewidziana klasa')
plt.tight_layout()
plt.show()

# Analiza błędów poszczególnych modeli bazowych
print("\n" + "="*70)
print("ANALIZA RÓŻNORODNOŚCI MODELI")
print("="*70)

# Trenuj modele bazowe
base_predictions = {}
for name, model in base_learners:
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    base_predictions[name] = pred
    print(f"\n{name} - Dokładność: {accuracy_score(y_test, pred):.4f}")

# Analiza zgodności predykcji
print("\nMacierz zgodności predykcji między modelami:")
agreement_matrix = np.zeros((len(base_learners), len(base_learners)))

for i, (name1, pred1) in enumerate(base_predictions.items()):
    for j, (name2, pred2) in enumerate(base_predictions.items()):
        agreement = np.mean(pred1 == pred2)
        agreement_matrix[i, j] = agreement

plt.figure(figsize=(8, 6))
sns.heatmap(agreement_matrix, annot=True, fmt='.3f',
            xticklabels=base_predictions.keys(),
            yticklabels=base_predictions.keys(),
            cmap='YlOrRd', vmin=0, vmax=1)
plt.title('Zgodność predykcji między modelami\n(1.0 = całkowita zgodność)')
plt.tight_layout()
plt.show()

print("\nWnioski:")
print("- Metody zespołowe często poprawiają wyniki, uzyskane za pomocą pojedynczych modeli")
print("- Bagging redukuje wariancję (działa najlepiej z niestabilnymi modelami)")
print("- Boosting redukuje zarówno obciążenie ("bias") jak i wariancję")
print("- Stacking może osiągnąć najlepsze wyniki, ale jest najbardziej złożony")
print("- Różnorodność modeli bazowych jest istotna dla metod zespołowych")
print("- Im mniejsza zgodność między modelami, tym większy potencjał ensemblingu")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-1076035410.py, line 359)

## Przykład 2: Regresja - Ensemble dla przewidywania wartości ciągłych

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# Ensemble methods
from sklearn.ensemble import (
    BaggingRegressor,
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
    StackingRegressor
)

import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("ENSEMBLING DLA REGRESJI")
print("="*70)

# Załadowanie danych
data = fetch_california_housing()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nProblem: Przewidywanie cen domów w Kalifornii")
print(f"Rozmiar zbioru treningowego: {X_train.shape}")
print(f"Rozmiar zbioru testowego: {X_test.shape}")
print(f"Zakres wartości docelowej: [{y.min():.2f}, {y.max():.2f}]")

# ============================================================================
# SEKCJA 1: MODELE BAZOWE
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 1: MODELE BAZOWE")
print("="*70)

base_models = {
    'Drzewo decyzyjne': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Regresja liniowa': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'SVR': SVR(kernel='rbf'),
    'KNN': KNeighborsRegressor(n_neighbors=5)
}

base_results = {}

for name, model in base_models.items():
    # Cross-validation RMSE
    scores = -cross_val_score(model, X_train, y_train, cv=5,
                              scoring='neg_root_mean_squared_error')
    base_results[name] = {
        'mean': scores.mean(),
        'std': scores.std()
    }
    print(f"\n{name}:")
    print(f"  CV RMSE: {scores.mean():.4f} (+/- {scores.std():.4f})")

# ============================================================================
# SEKCJA 2: BAGGING ENSEMBLE
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 2: BAGGING - STABILIZACJA PREDYKCJI")
print("="*70)

# Bagging z Decision Tree
bagging = BaggingRegressor(
    estimator=DecisionTreeRegressor(max_depth=10, random_state=42),
    n_estimators=50,
    max_samples=0.8,
    max_features=0.8,
    random_state=42
)

scores_bagging = -cross_val_score(bagging, X_train, y_train, cv=5,
                                  scoring='neg_root_mean_squared_error')
print(f"\nBagging (drzewo decyzyjne, n=50):")
print(f"  CV RMSE: {scores_bagging.mean():.4f} (+/- {scores_bagging.std():.4f})")
print(f"  Redukcja RMSE vs drzewo: {base_results['Drzewo decyzyjne']['mean'] - scores_bagging.mean():.4f}")

# Random Forest
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    random_state=42
)

scores_rf = -cross_val_score(rf, X_train, y_train, cv=5,
                             scoring='neg_root_mean_squared_error')
print(f"\nRandom Forest (n=100):")
print(f"  CV RMSE: {scores_rf.mean():.4f} (+/- {scores_rf.std():.4f})")

# Analiza feature importance
rf.fit(X_train, y_train)
feature_importance = pd.DataFrame({
    'cecha': data.feature_names,
    'waznosc': rf.feature_importances_
}).sort_values('waznosc', ascending=False)

print(f"\nTop 5 najważniejszych cech (Random Forest):")
print(feature_importance.head())

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['cecha'], feature_importance['waznosc'])
plt.xlabel('Ważność cechy')
plt.title('Ważność cech w Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# ============================================================================
# SEKCJA 3: BOOSTING ENSEMBLE
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 3: BOOSTING - SEKWENCYJNE UCZENIE")
print("="*70)

# AdaBoost
adaboost = AdaBoostRegressor(
    estimator=DecisionTreeRegressor(max_depth=5),
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
)

scores_ada = -cross_val_score(adaboost, X_train, y_train, cv=5,
                              scoring='neg_root_mean_squared_error')
print(f"\nAdaBoost (n=100, lr=0.5):")
print(f"  CV RMSE: {scores_ada.mean():.4f} (+/- {scores_ada.std():.4f})")

# Gradient Boosting
gb = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    random_state=42
)

scores_gb = -cross_val_score(gb, X_train, y_train, cv=5,
                             scoring='neg_root_mean_squared_error')
print(f"\nGradient Boosting (n=100, lr=0.1):")
print(f"  CV RMSE: {scores_gb.mean():.4f} (+/- {scores_gb.std():.4f})")

# Analiza wpływu liczby iteracji na Gradient Boosting
print("\nAnaliza błędu treningowego vs walidacyjnego (Gradient Boosting):")

gb_analyze = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    random_state=42
)

gb_analyze.fit(X_train, y_train)

# Błędy w funkcji liczby iteracji
train_scores = []
val_scores = []

for i, (train_pred, val_pred) in enumerate(zip(
    gb_analyze.staged_predict(X_train),
    gb_analyze.staged_predict(X_test)
)):
    train_scores.append(mean_squared_error(y_train, train_pred))
    val_scores.append(mean_squared_error(y_test, val_pred))

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_scores) + 1), train_scores,
         label='Błąd treningowy', linewidth=2)
plt.plot(range(1, len(val_scores) + 1), val_scores,
         label='Błąd walidacyjny', linewidth=2)
plt.xlabel('Liczba iteracji')
plt.ylabel('MSE')
plt.title('Krzywa uczenia Gradient Boosting')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

optimal_iterations = np.argmin(val_scores) + 1
print(f"Optymalna liczba iteracji: {optimal_iterations}")
print(f"MSE przy optymalnej liczbie iteracji: {val_scores[optimal_iterations-1]:.4f}")

# ============================================================================
# SEKCJA 4: VOTING ENSEMBLE
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 4: VOTING - UŚREDNIANIE PREDYKCJI")
print("="*70)

# Voting Regressor - średnia z różnych modeli
voting = VotingRegressor(
    estimators=[
        ('rf', RandomForestRegressor(n_estimators=50, random_state=42)),
        ('gb', GradientBoostingRegressor(n_estimators=50, random_state=42)),
        ('ridge', Ridge(alpha=1.0))
    ]
)

scores_voting = -cross_val_score(voting, X_train, y_train, cv=5,
                                 scoring='neg_root_mean_squared_error')
print(f"\nVoting (RF + GB + Ridge):")
print(f"  CV RMSE: {scores_voting.mean():.4f} (+/- {scores_voting.std():.4f})")

# Voting z wagami
voting_weighted = VotingRegressor(
    estimators=[
        ('rf', RandomForestRegressor(n_estimators=50, random_state=42)),
        ('gb', GradientBoostingRegressor(n_estimators=50, random_state=42)),
        ('ridge', Ridge(alpha=1.0))
    ],
    weights=[2, 3, 1]  # Większa waga dla Gradient Boosting
)

scores_voting_weighted = -cross_val_score(voting_weighted, X_train, y_train, cv=5,
                                         scoring='neg_root_mean_squared_error')
print(f"\nVoting Weighted (wagi: RF=2, GB=3, Ridge=1):")
print(f"  CV RMSE: {scores_voting_weighted.mean():.4f} (+/- {scores_voting_weighted.std():.4f})")

# ============================================================================
# SEKCJA 5: STACKING ENSEMBLE
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 5: STACKING - META-REGRESSOR")
print("="*70)

# Modele bazowe
base_learners = [
    ('rf', RandomForestRegressor(n_estimators=50, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=50, random_state=42)),
    ('ridge', Ridge(alpha=1.0)),
    ('svr', SVR(kernel='rbf', C=1.0))
]

# Meta-model
stacking = StackingRegressor(
    estimators=base_learners,
    final_estimator=Ridge(alpha=0.5),
    cv=5
)

scores_stacking = -cross_val_score(stacking, X_train, y_train, cv=5,
                                   scoring='neg_root_mean_squared_error')
print(f"\nStacking (RF + GB + Ridge + SVR → Ridge):")
print(f"  CV RMSE: {scores_stacking.mean():.4f} (+/- {scores_stacking.std():.4f})")

# ============================================================================
# SEKCJA 6: PORÓWNANIE I WIZUALIZACJA
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 6: PORÓWNANIE WSZYSTKICH METOD")
print("="*70)

all_results = {
    'Drzewo decyzyjne': base_results['Drzewo decyzyjne']['mean'],
    'Regresja liniowa': base_results['Regresja liniowa']['mean'],
    'Ridge': base_results['Ridge']['mean'],
    'Bagging': scores_bagging.mean(),
    'Random Forest': scores_rf.mean(),
    'AdaBoost': scores_ada.mean(),
    'Gradient Boosting': scores_gb.mean(),
    'Voting': scores_voting.mean(),
    'Voting Weighted': scores_voting_weighted.mean(),
    'Stacking': scores_stacking.mean()
}

sorted_results = dict(sorted(all_results.items(), key=lambda x: x[1]))

print("\nRanking metod (od najlepszej - najniższy RMSE):")
for i, (method, rmse) in enumerate(sorted_results.items(), 1):
    print(f"{i:2d}. {method:20s}: RMSE = {rmse:.4f}")

# Wizualizacja
plt.figure(figsize=(12, 8))
methods = list(sorted_results.keys())
rmses = list(sorted_results.values())
colors = ['red' if any(x in m for x in ['Drzewo', 'liniowa', 'Ridge'])
          else 'green' if any(x in m for x in ['Bagging', 'Random', 'AdaBoost', 'Gradient'])
          else 'blue' for m in methods]

plt.barh(methods, rmses, color=colors, alpha=0.7)
plt.xlabel('RMSE (niższy = lepszy)')
plt.title('Porównanie metod ensemblingu dla regresji')
for i, (method, rmse) in enumerate(zip(methods, rmses)):
    plt.text(rmse + 0.01, i, f'{rmse:.4f}', va='center', fontweight='bold')
plt.tight_layout()
plt.show()

# ============================================================================
# SEKCJA 7: SZCZEGÓŁOWA OCENA NAJLEPSZEGO MODELU
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 7: SZCZEGÓŁOWA OCENA NAJLEPSZEGO MODELU")
print("="*70)

# Trenuj najlepszy model
best_model = stacking
best_model.fit(X_train, y_train)

# Predykcje
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print(f"\nNajlepszy model: Stacking Regressor")
print(f"\nWyniki na zbiorze treningowym:")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_train, y_train_pred):.4f}")
print(f"  R²: {r2_score(y_train, y_train_pred):.4f}")

print(f"\nWyniki na zbiorze testowym:")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")
print(f"  R²: {r2_score(y_test, y_test_pred):.4f}")

# Wizualizacja predykcji
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot
axes[0].scatter(y_test, y_test_pred, alpha=0.5, s=30)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
             'r--', lw=2, label='Idealna predykcja')
axes[0].set_xlabel('Rzeczywiste wartości')
axes[0].set_ylabel('Przewidywane wartości')
axes[0].set_title('Predykcje vs Rzeczywiste wartości')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residual plot
residuals = y_test - y_test_pred
axes[1].scatter(y_test_pred, residuals, alpha=0.5, s=30)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Przewidywane wartości')
axes[1].set_ylabel('Residuals (błędy)')
axes[1].set_title('Wykres residuów')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analiza błędów modeli bazowych vs ensemble
print("\n" + "="*70)
print("ANALIZA KOMPLEMENTARNOŚCI MODELI")
print("="*70)

# Predykcje modeli bazowych
base_predictions = {}
for name, model in base_learners:
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    base_predictions[name] = pred
    print(f"\n{name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  Korelacja z residuami innych modeli:")

    for name2, pred2 in base_predictions.items():
        if name != name2:
            residuals2 = y_test - pred2
            correlation = np.corrcoef(pred, residuals2)[0, 1]
            print(f"    vs {name2}: {correlation:.4f}")

print("\nWnioski:")
print("- Metody zespołowe redukują błąd predykcji w porównaniu do pojedynczych modeli")
print("- Gradient Boosting często osiąga najlepsze wyniki dla regresji")
print("- Stacking może dalej poprawić wyniki łącząc różnorodne modele")
print("- Voting jest prostszy w implementacji i daje dobre rezultaty")
print("- Modele bazowe powinny mieć niską korelację błędów dla efektywnego ensemblingu")

## Przykład 3: Klasteryzacja metodami zespołowymi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.stats import mode
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("ENSEMBLE CLUSTERING")
print("="*70)

# Generowanie danych
np.random.seed(42)
X, y_true = make_blobs(
    n_samples=600,
    n_features=2,
    centers=4,
    cluster_std=1.2,
    random_state=42
)

# Dodaj trochę szumu
noise = np.random.normal(0, 0.5, X.shape)
X = X + noise

# Skalowanie
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nRozmiar danych: {X.shape}")
print(f"Liczba prawdziwych klastrów: {len(np.unique(y_true))}")

# Wizualizacja danych
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=50, alpha=0.6, edgecolors='black')
plt.title('Dane wejściowe (prawdziwe klastry)')
plt.xlabel('Cecha 1')
plt.ylabel('Cecha 2')
plt.colorbar(label='Klaster')
plt.tight_layout()
plt.show()

# ============================================================================
# SEKCJA 1: POJEDYNCZE ALGORYTMY KLASTERYZACJI
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 1: POJEDYNCZE ALGORYTMY")
print("="*70)

n_clusters = 4

# KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
labels_kmeans = kmeans.fit_predict(X_scaled)

# Hierarchical Clustering (różne linkaże)
agg_ward = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
labels_ward = agg_ward.fit_predict(X_scaled)

agg_average = AgglomerativeClustering(n_clusters=n_clusters, linkage='average')
labels_average = agg_average.fit_predict(X_scaled)

agg_complete = AgglomerativeClustering(n_clusters=n_clusters, linkage='complete')
labels_complete = agg_complete.fit_predict(X_scaled)

# DBSCAN (wymaga innych parametrów)
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels_dbscan = dbscan.fit_predict(X_scaled)

# Ocena pojedynczych algorytmów
algorithms = {
    'KMeans': labels_kmeans,
    'Hierarchiczny (Ward)': labels_ward,
    'Hierarchiczny (Average)': labels_average,
    'Hierarchiczny (Complete)': labels_complete,
    'DBSCAN': labels_dbscan
}

print("\nOcena pojedynczych algorytmów:")
for name, labels in algorithms.items():
    if len(np.unique(labels)) > 1:  # DBSCAN może znaleźć 1 klaster
        silhouette = silhouette_score(X_scaled, labels)
        davies_bouldin = davies_bouldin_score(X_scaled, labels)
        ari = adjusted_rand_score(y_true, labels)
        print(f"\n{name}:")
        print(f"  Liczba klastrów: {len(np.unique(labels))}")
        print(f"  Silhouette: {silhouette:.4f}")
        print(f"  Davies-Bouldin: {davies_bouldin:.4f}")
        print(f"  ARI (vs prawdziwe): {ari:.4f}")
    else:
        print(f"\n{name}: Znaleziono tylko 1 klaster")

# Wizualizacja wyników pojedynczych algorytmów
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

# Prawdziwe klastry
axes[0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=50, alpha=0.6, edgecolors='black')
axes[0].set_title('Prawdziwe klastry')

# Algorytmy
for i, (name, labels) in enumerate(algorithms.items(), 1):
    axes[i].scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=50, alpha=0.6, edgecolors='black')
    axes[i].set_title(name)

plt.tight_layout()
plt.show()

# ============================================================================
# SEKCJA 2: CONSENSUS CLUSTERING (Voting)
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 2: CONSENSUS CLUSTERING - GŁOSOWANIE")
print("="*70)

print("\nMetoda: Consensus clustering poprzez co-association matrix")
print("Idea: Zliczamy, jak często pary punktów są w tym samym klastrze")

# Użyjemy tylko algorytmów, które znalazły więcej niż 1 klaster
valid_algorithms = {k: v for k, v in algorithms.items() if len(np.unique(v)) > 1}

# Tworzenie macierzy współwystępowania
n_samples = X_scaled.shape[0]
n_algorithms = len(valid_algorithms)
co_association_matrix = np.zeros((n_samples, n_samples))

for name, labels in valid_algorithms.items():
    for i in range(n_samples):
        for j in range(i+1, n_samples):
            if labels[i] == labels[j]:
                co_association_matrix[i, j] += 1
                co_association_matrix[j, i] += 1

# Normalizacja
co_association_matrix /= n_algorithms

print(f"\nMacierz współwystępowania obliczona z {n_algorithms} algorytmów")
print(f"Średnia wartość w macierzy: {co_association_matrix.mean():.4f}")

# Konwersja macierzy współwystępowania na macierz odległości
distance_matrix = 1 - co_association_matrix

# Hierarchical clustering na macierzy odległości
consensus_clustering = AgglomerativeClustering(
    n_clusters=n_clusters,
    metric='precomputed',
    linkage='average'
)
labels_consensus = consensus_clustering.fit_predict(distance_matrix)

# Ocena
silhouette_consensus = silhouette_score(X_scaled, labels_consensus)
davies_bouldin_consensus = davies_bouldin_score(X_scaled, labels_consensus)
ari_consensus = adjusted_rand_score(y_true, labels_consensus)

print(f"\nConsensus Clustering:")
print(f"  Silhouette: {silhouette_consensus:.4f}")
print(f"  Davies-Bouldin: {davies_bouldin_consensus:.4f}")
print(f"  ARI (vs prawdziwe): {ari_consensus:.4f}")

# ============================================================================
# SEKCJA 3: ENSEMBLE Z RÓŻNYMI INICJALIZACJAMI
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 3: ENSEMBLE Z RÓŻNYMI INICJALIZACJAMI")
print("="*70)

print("\nMetoda: Uruchom KMeans wielokrotnie z różnymi inicjalizacjami")

n_runs = 20
kmeans_ensemble_labels = []

for i in range(n_runs):
    km = KMeans(n_clusters=n_clusters, random_state=i, n_init=1)
    labels = km.fit_predict(X_scaled)
    kmeans_ensemble_labels.append(labels)

# Macierz współwystępowania dla ensemble KMeans
co_assoc_kmeans = np.zeros((n_samples, n_samples))

for labels in kmeans_ensemble_labels:
    for i in range(n_samples):
        for j in range(i+1, n_samples):
            if labels[i] == labels[j]:
                co_assoc_kmeans[i, j] += 1
                co_assoc_kmeans[j, i] += 1

co_assoc_kmeans /= n_runs

# Consensus clustering
distance_kmeans = 1 - co_assoc_kmeans
ensemble_kmeans = AgglomerativeClustering(
    n_clusters=n_clusters,
    metric='precomputed',
    linkage='average'
)
labels_ensemble_kmeans = ensemble_kmeans.fit_predict(distance_kmeans)

# Ocena
silhouette_ens_km = silhouette_score(X_scaled, labels_ensemble_kmeans)
davies_bouldin_ens_km = davies_bouldin_score(X_scaled, labels_ensemble_kmeans)
ari_ens_km = adjusted_rand_score(y_true, labels_ensemble_kmeans)

print(f"\nEnsemble KMeans ({n_runs} uruchomień):")
print(f"  Silhouette: {silhouette_ens_km:.4f}")
print(f"  Davies-Bouldin: {davies_bouldin_ens_km:.4f}")
print(f"  ARI (vs prawdziwe): {ari_ens_km:.4f}")

# ============================================================================
# SEKCJA 4: ENSEMBLE Z RÓŻNYMI PODZBIORAMI CECH
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 4: ENSEMBLE Z RÓŻNYMI PODZBIORAMI CECH")
print("="*70)

print("\nMetoda: Klasteryzacja na losowych podzbiorach cech (Feature Bagging)")
print("Symulacja: Dodamy dodatkowe cechy szumu")

# Dodaj więcej wymiarów (cechy szumu)
np.random.seed(42)
X_extended = np.hstack([X_scaled, np.random.randn(n_samples, 8)])

print(f"Rozszerzony zbiór danych: {X_extended.shape[1]} cech")

n_feature_subsets = 15
feature_subset_size = 5
feature_ensemble_labels = []

for i in range(n_feature_subsets):
    # Losowy podzbiór cech
    np.random.seed(i)
    feature_indices = np.random.choice(X_extended.shape[1],
                                      feature_subset_size,
                                      replace=False)
    X_subset = X_extended[:, feature_indices]

    # KMeans na podzbiorze
    km = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = km.fit_predict(X_subset)
    feature_ensemble_labels.append(labels)

# Consensus clustering
co_assoc_features = np.zeros((n_samples, n_samples))

for labels in feature_ensemble_labels:
    for i in range(n_samples):
        for j in range(i+1, n_samples):
            if labels[i] == labels[j]:
                co_assoc_features[i, j] += 1
                co_assoc_features[j, i] += 1

co_assoc_features /= n_feature_subsets

distance_features = 1 - co_assoc_features
ensemble_features = AgglomerativeClustering(
    n_clusters=n_clusters,
    metric='precomputed',
    linkage='average'
)
labels_ensemble_features = ensemble_features.fit_predict(distance_features)

# Ocena
silhouette_ens_feat = silhouette_score(X_scaled, labels_ensemble_features)
davies_bouldin_ens_feat = davies_bouldin_score(X_scaled, labels_ensemble_features)
ari_ens_feat = adjusted_rand_score(y_true, labels_ensemble_features)

print(f"\nFeature Bagging Ensemble ({n_feature_subsets} podzbiorów, {feature_subset_size} cech każdy):")
print(f"  Silhouette: {silhouette_ens_feat:.4f}")
print(f"  Davies-Bouldin: {davies_bouldin_ens_feat:.4f}")
print(f"  ARI (vs prawdziwe): {ari_ens_feat:.4f}")

# ============================================================================
# SEKCJA 5: PORÓWNANIE WSZYSTKICH METOD
# ============================================================================
print("\n" + "="*70)
print("SEKCJA 5: PORÓWNANIE WSZYSTKICH METOD")
print("="*70)

all_results = {
    'KMeans (pojedynczy)': {
        'silhouette': silhouette_score(X_scaled, labels_kmeans),
        'ari': adjusted_rand_score(y_true, labels_kmeans),
        'labels': labels_kmeans
    },
    'Hierarchiczny Ward': {
        'silhouette': silhouette_score(X_scaled, labels_ward),
        'ari': adjusted_rand_score(y_true, labels_ward),
        'labels': labels_ward
    },
    'Consensus (algorytmy)': {
        'silhouette': silhouette_consensus,
        'ari': ari_consensus,
        'labels': labels_consensus
    },
    'Ensemble KMeans': {
        'silhouette': silhouette_ens_km,
        'ari': ari_ens_km,
        'labels': labels_ensemble_kmeans
    },
    'Feature Bagging': {
        'silhouette': silhouette_ens_feat,
        'ari': ari_ens_feat,
        'labels': labels_ensemble_features
    }
}

print("\nRanking według Silhouette Score:")
sorted_by_silhouette = sorted(all_results.items(),
                             key=lambda x: x[1]['silhouette'],
                             reverse=True)
for i, (method, scores) in enumerate(sorted_by_silhouette, 1):
    print(f"{i}. {method:30s}: Silhouette={scores['silhouette']:.4f}, ARI={scores['ari']:.4f}")

# Wizualizacja porównawcza
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, (method, result) in enumerate(all_results.items()):
    axes[i].scatter(X[:, 0], X[:, 1], c=result['labels'],
                   cmap='viridis', s=50, alpha=0.6, edgecolors='black')
    axes[i].set_title(f"{method}\nSilhouette: {result['silhouette']:.4f}, ARI: {result['ari']:.4f}")
    axes[i].set_xlabel('Cecha 1')
    axes[i].set_ylabel('Cecha 2')

# Prawdziwe klastry dla porównania
axes[5].scatter(X[:, 0], X[:, 1], c=y_true,
               cmap='viridis', s=50, alpha=0.6, edgecolors='black')
axes[5].set_title('Prawdziwe klastry')
axes[5].set_xlabel('Cecha 1')
axes[5].set_ylabel('Cecha 2')

plt.tight_layout()
plt.show()

# Porównanie metryk
methods = list(all_results.keys())
silhouettes = [all_results[m]['silhouette'] for m in methods]
aris = [all_results[m]['ari'] for m in methods]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].barh(methods, silhouettes, color='steelblue', alpha=0.7)
axes[0].set_xlabel('Silhouette Score')
axes[0].set_title('Silhouette Score (wyższy = lepszy)')
for i, v in enumerate(silhouettes):
    axes[0].text(v + 0.005, i, f'{v:.4f}', va='center')

axes[1].barh(methods, aris, color='coral', alpha=0.7)
axes[1].set_xlabel('Adjusted Rand Index')
axes[1].set_title('ARI vs prawdziwe klastry (wyższy = lepszy)')
for i, v in enumerate(aris):
    axes[1].text(v + 0.005, i, f'{v:.4f}', va='center')

plt.tight_layout()
plt.show()

# Wizualizacja macierzy współwystępowania
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Consensus algorytmów
im1 = axes[0].imshow(co_association_matrix, cmap='YlOrRd', vmin=0, vmax=1)
axes[0].set_title('Co-Association Matrix\n(Consensus algorytmów)')
plt.colorbar(im1, ax=axes[0])

# Ensemble KMeans
im2 = axes[1].imshow(co_assoc_kmeans, cmap='YlOrRd', vmin=0, vmax=1)
axes[1].set_title('Co-Association Matrix\n(Ensemble KMeans)')
plt.colorbar(im2, ax=axes[1])

# Feature Bagging
im3 = axes[2].imshow(co_assoc_features, cmap='YlOrRd', vmin=0, vmax=1)
axes[2].set_title('Co-Association Matrix\n(Feature Bagging)')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

print("\nWnioski:")
print("- Klasteryzacja zespołowa na ogół zwiększa stabilność i jakość klasteryzacji")
print("- 'Consensus clustering' łączy mocne strony różnych algorytmów")
print("- Wielokrotne uruchomienia z różnymi inicjalizacjami redukują wpływ losowości")
print("- 'Feature bagging' pomaga, gdy mamy wiele cech, z których część może być szumem")
print("- 'Co-association matrix' wizualizuje pewność przypisania do klastrów")
print("- Metody zespołowe są szczególnie przydatne gdy struktura klastrów jest niejednoznaczna")